In [3]:
import boto3
import os
import re
from ipython_secrets import * 


Downloaded ds000030/ds000030_R1.0.5/uncompressed/derivatives/fmriprep/sub-10159/func/sub-10159_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz to C:\Users\stapl\Documents\CDocuments\FinalYearProject\Model\nifti_files\sub-10159_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
Downloaded ds000030/ds000030_R1.0.5/uncompressed/derivatives/fmriprep/sub-10159/func/sub-10159_task-rest_bold_confounds.tsv to C:\Users\stapl\Documents\CDocuments\FinalYearProject\Model\confounds\sub-10159_task-rest_bold_confounds.tsv


To get confounds and resting BOLD 

In [ ]:

# The get_secret function from ipython_secrets only requests the keys once before storing them in the Jupyter notebook's environment
aws_api_key = get_secret("aws_api_key")
aws_api_secret = get_secret("aws_api_secret")

# Create an S3 client using the provided access keys
client = boto3.client('s3', 
                      aws_access_key_id=aws_api_key, 
                      aws_secret_access_key=aws_api_secret)

bucket_name = 'openneuro'
key = 'ds000030/ds000030_R1.0.5/uncompressed/derivatives/'

response = client.list_objects_v2(Bucket=bucket_name, Prefix=key)

local_dir = r'C:\Users\stapl\Documents\CDocuments\FinalYearProject\Model'

subjects = [
    'sub-10159'
    # 'sub-70004',... etc 
    # Relevant files do not exist for - sub-10948, sub-11082, sub-70002 
]

pattern_nifti = r"sub-\d{5}_task-rest_bold_space-MNI152NLin2009cAsym_preproc\.nii\.gz"
pattern_confound = r"sub-\d{5}_task-rest_bold_confounds\.tsv"

nifti_dir = os.path.join(local_dir, 'nifti_files')
confound_dir = os.path.join(local_dir, 'confounds')

# Iterate over subjects and download files
for subject in subjects:
    # Construct the keys for the files for the current subject
    nifti_key = f"ds000030/ds000030_R1.0.5/uncompressed/derivatives/fmriprep/{subject}/func/{subject}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz"
    confound_key = f"ds000030/ds000030_R1.0.5/uncompressed/derivatives/fmriprep/{subject}/func/{subject}_task-rest_bold_confounds.tsv"
    
    # Define the local file paths to save the downloaded files
    local_nifti_file = os.path.join(nifti_dir, f"{subject}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz")
    local_confound_file = os.path.join(confound_dir, f"{subject}_task-rest_bold_confounds.tsv")
    
    # Download the NIFTI file
    client.download_file(bucket_name, nifti_key, local_nifti_file)
    print(f"Downloaded {nifti_key} to {local_nifti_file}")
    
    # Download the confound file
    client.download_file(bucket_name, confound_key, local_confound_file)
    print(f"Downloaded {confound_key} to {local_confound_file}")

To get sMRI 

In [19]:
# The get_secret function from ipython_secrets only requests the keys once before storing them in the Jupyter notebook's environment
aws_api_key = get_secret("aws_api_key")
aws_api_secret = get_secret("aws_api_secret")

# Create an S3 client using the provided access keys
client = boto3.client('s3', 
                      aws_access_key_id=aws_api_key, 
                      aws_secret_access_key=aws_api_secret)

bucket_name = 'openneuro'
key = 'ds000030/ds000030_R1.0.5/uncompressed/derivatives/fmriprep/'

local_dir = r'C:\Users\stapl\Documents\CDocuments\FinalYearProject\Model'

subjects = [
    'sub-10159'
    # 'sub-70004',... etc 
    # Relevant files do not exist for - sub-10948, sub-11082, sub-70002 
]

sMRI_dir = os.path.join(local_dir, 'sMRI')

# Iterate over subjects and download files
for subject in subjects:
    # Construct the keys for the files for the current subject
    
    sMRI_key = f"ds000030/ds000030_R1.0.5/uncompressed/derivatives/fmriprep/{subject}/anat/{subject}_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz"

    # Define the local file paths to save the downloaded files
    local_sMRI_file = os.path.join(sMRI_dir, f"{subject}_T1w.nii.gz")
    
    # Download the NIFTI file
    client.download_file(bucket_name, sMRI_key, local_sMRI_file)
    print(f"Downloaded {sMRI_key} to {local_sMRI_file}")


Downloaded ds000030/ds000030_R1.0.5/uncompressed/derivatives/fmriprep/sub-10159/anat/sub-10159_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz to C:\Users\stapl\Documents\CDocuments\FinalYearProject\Model\sMRI\sub-10159_T1w.nii.gz


To get the raw EEG data

In [1]:
import mne
import os 

eeg_dir = 'eeg_data/Raw_Subset/'
files = os.listdir(eeg_dir)
set_files = [f for f in files if f.endswith('.set')]

all_psd_emp = [] # This will have 39 entries 

# To ignore warnings on read_raw_eeglab
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

for set_file in set_files:
    if "sasica" in set_file.lower():
        eeg_path = os.path.join(eeg_dir, set_file)
        raw = mne.io.read_raw_eeglab(eeg_path)

eeg_raw_dir = 'C:/Users/stapl/Documents/CDocuments/FinalYearProject/Model/eeg_raw_data'
# Doing this twice as storing the raw file causes a memory error
for set_file in set_files:
    if "sasica" in set_file.lower():
        eeg_path = os.path.join(eeg_dir, set_file)
        raw = mne.io.read_raw_eeglab(eeg_path)
        eeg_raw_file = os.path.join(eeg_raw_dir, os.path.splitext(set_file)[0] + '_preprocessed-raw.fif')
        raw.save(eeg_raw_file, overwrite=True)

c:\Users\stapl\anaconda3\envs\nilearnenv\lib\site-packages\pymatreader\utils.py:124: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


Reading c:\Users\stapl\Documents\CDocuments\FinalYearProject\Model\eeg_data\Raw_Subset\filt_chanrej_SASICA_MARA_0162_081610_RestEyesClosed_DvP_1.fdt


c:\Users\stapl\anaconda3\envs\nilearnenv\lib\site-packages\pymatreader\utils.py:124: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


Reading c:\Users\stapl\Documents\CDocuments\FinalYearProject\Model\eeg_data\Raw_Subset\filt_chanrej_SASICA_MARA_0234_102011_RestEyesClosed_DVP_1.fdt
Reading c:\Users\stapl\Documents\CDocuments\FinalYearProject\Model\eeg_data\Raw_Subset\filt_chanrej_SASICA_MARA_0332_042410_RestEyesClosed_RS_1.fdt
Reading c:\Users\stapl\Documents\CDocuments\FinalYearProject\Model\eeg_data\Raw_Subset\filt_chanrej_SASICA_MARA_0587_030111_RestEyesClosed_CN_1.fdt
Reading c:\Users\stapl\Documents\CDocuments\FinalYearProject\Model\eeg_data\Raw_Subset\filt_chanrej_SASICA_MARA_0632_072510_RestEyesClosed_AG_1.fdt
Reading c:\Users\stapl\Documents\CDocuments\FinalYearProject\Model\eeg_data\Raw_Subset\filt_chanrej_SASICA_MARA_1536_081810_RestEyesClosed_DP_1.fdt
Reading c:\Users\stapl\Documents\CDocuments\FinalYearProject\Model\eeg_data\Raw_Subset\filt_chanrej_SASICA_MARA_1557_101410_RestEyesClosed_KO_1.fdt
Reading c:\Users\stapl\Documents\CDocuments\FinalYearProject\Model\eeg_data\Raw_Subset\filt_chanrej_SASICA_MARA

To transform the leadfield from 100 62x3 matrices into a 100,62,3 leadfield

In [ ]:
import scipy.io
import numpy as np

mat_data = scipy.io.loadmat('leadfield.mat')
leadfield = mat_data['leadfield']

array = leadfield[0][0][4][0]

print(array[0].shape)  # should output (62, 3)

reshaped_array = np.array([x for x in array])
reshaped_array = reshaped_array.reshape(100, 62, 3)

print(reshaped_array.shape)  # should output (100, 62, 3)

scipy.io.savemat('reshaped_leadfield.mat', {'leadfield': reshaped_array})